In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import pandas as pd
from sklearn.model_selection import train_test_split
import os
print("Tensorflow version:", tf.version.VERSION)

In [ ]:
df = pd.read_csv('data.csv')
X = df[['pressure', 'temperature', 'humidity']].values
y = df[['rain']].values

In [ ]:
def mymodel(X,y):

    tf.random.set_seed(seed=0)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)
    
    # Create Keras model
    model = keras.Sequential(layers=[
        keras.layers.InputLayer(input_shape=(3), name="input"),
        keras.layers.Dense(3, activation="relu", name="dense"),
        keras.layers.Dense(1, activation="sigmoid", name="output")
    ],
                             name="mymodel")

    # Print model architecture
    model.summary()

    # Compile model with optimizer
    model.compile(optimizer=keras.optimizers.Adam(0.001),
                  loss="binary_crossentropy",
                  metrics=["accuracy"])

    # Train model
    model.fit(x=[X_train], y=[y_train], batch_size=100, epochs=10)

    # Test model
    test_loss, test_acc = model.evaluate(x=[X_test],
                                         y=[y_test],
                                         verbose=2)
    print("-" * 50)
    print("Test accuracy: ")
    print(test_acc)

    # Get predictions for test images
    predictions = model.predict(X_test)
    # Print the prediction for the first image
    print("-" * 50)
    print("Example prediction reference:")
    print(predictions)

    # Save model to SavedModel format
    tf.saved_model.save(model, "./models")

    return model

In [ ]:
model = mymodel(X, y)

In [ ]:
imported = tf.saved_model.load('models')


In [ ]:
print(list(imported.signatures.keys()))

In [ ]:
infer = imported.signatures["serving_default"]

In [ ]:
infer(tf.constant(X, dtype=float))

In [ ]:
model.__call__.get_concrete_function(tf.TensorSpec(None, tf.float32))


# Create a Servable ML Model with Tensorflow
In this notebook we will create, save, load, and employ models with Tensorflow. We will work through how to structure code to create models that can be saved and used for inference in the cloud or at the edge with applications such as dashboards, games, anomaly detection, and much more.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import pandas as pd
from sklearn.model_selection import train_test_split
import os
print("Tensorflow version:", tf.version.VERSION)
print("Numpy version:", np.version.version)

Tensorflow version: 2.1.0
Numpy version: 1.18.1


In [2]:
MODEL_DIR = 'models' # We will save model to this directory
DATA_PATH = os.path.join('data', 'data.csv') 

Tensorflow 2 includes a [SavedModel](https://www.tensorflow.org/guide/saved_model "Tensorflow SavedModel docs") format that can be utilised for transfer learning and/or inference in applications. In order for a model to be saveable, it has to be of the type `tf.Module`. Keras models satisfy this criterium and are thus relatively simple to save. Before jumping into a specific example of a Keras model, we will, however, address the components of a Tensorflow SavedModel with a general example using pure Tensorflow.

# General Example in Pure Tensorflow
A Tensorflow SavedModel can be created from a `tf.Module`, so let us create a very simple example model using this object structure. The saved model will include any `tf.Modules`s, any methods with the `@tf.function` decorator, and any `tf.Variable`, but will not include any Python code or functionality.

In [55]:
class LinearScaler(tf.Module):
    '''
    LinearScaler is a very simple linear function that takes in a variable, multiplies it
     by a weight and adds a bias before returning the result.
    '''
    def __init__(self):
        super(LinearScaler, self).__init__()
        self.bias = tf.Variable(1.)
        self.weight = tf.Variable(2.)

    @tf.function#(input_signature=[tf.TensorSpec([], tf.float32)])
    def __call__(self, x):
        '''
        Linearly rescale y = x * weight + bias
        :param x: The variable to be linearly scaled
        :type x: tf.float32
        :output y: y = x * weight + bias
        '''
        return x * self.weight + self.bias

    @tf.function(input_signature=[tf.TensorSpec([], tf.float32), tf.TensorSpec([], tf.float32)])
    def calibrate(self, weight, bias):
        '''
        Set the parameters of the linear function.
        :param weight: Weight of the linear function
        :type weight: tf.float32
        :param bias: Bias of the linear function
        :type bias: tf.float32
        '''
        self.weight.assign(weight)
        self.bias.assign(bias)

model = LinearScaler()

Let us make sure that is works

In [56]:
assert model(4).numpy() == 9 #  2 * 4 + 1
model.calibrate(weight=5, bias=2)
assert model(4).numpy() == 22 # 5 * 4 + 2

Fantastic! Now we have a working model. Let us go right ahead and save it

In [57]:
no_signatures_path = os.path.join(MODEL_DIR, 'no_signatures')
#model = LinearScaler() # Get a fresh model
#model(tf.constant(4.))
tf.saved_model.save(model, no_signatures_path) # Save the function to a dir

INFO:tensorflow:Assets written to: models\no_signatures\assets


So that is our model saved. Let us have a look at what is in the directory

In [41]:
os.listdir(no_signatures_path)

['assets', 'saved_model.pb', 'variables']

The saved model consists of several elements.<br>
- `saved_model.pb` contains the model architecture that is used to rebuild the function
- The directory `variables` contain one or more data files holding the values of the parameters in the model at the time it was saved. For large models with billions of parameters, these data files can grow large. A `variables.index` file maps the stored parameters to their right spot in the function
- The directory `assets` holds additional artefacts needed to recreate the function, but should be empty in our case

Fortunately, it is fairly straightforward to load the model again, as the `tf.saved_model.load` method handles everything for us

In [58]:
loaded_model = tf.saved_model.load(no_signatures_path)

However, if we try to use our model in the same way as the original, we are presented with a ValueError

In [65]:
try:
    print(loaded_model(4))
except ValueError as e:
    print("We got a ValueError:\n", e)

tf.Tensor(22.0, shape=(), dtype=float32)


The loaded model is not as forgiving as our original model. Specifically, it will accept only tensor inputs and will return tensor outputs

In [63]:
loaded_model(2)

ValueError: Could not find matching function to call loaded from the SavedModel. Got:
  Positional arguments (1 total):
    * 2
  Keyword arguments: {}

Expected these arguments to match one of the following 1 option(s):

Option 1:
  Positional arguments (1 total):
    * 4
  Keyword arguments: {}

In [6]:
with_signature_path = os.path.join(MODEL_DIR, 'with_signature')
call = model.__call__.get_concrete_function(tf.TensorSpec(None, tf.float32)) # Get a one-stop function
tf.saved_model.save(model, with_signature_path, signatures=call) # Save the function

INFO:tensorflow:Assets written to: models\with_signature\assets


Okay, so there is a lot to unpack here

In [ ]:
imported = tf.saved_model.load(module_with_signature_path)
print(list(imported.signatures.keys()))

In [ ]:
infer = imported.signatures["serving_default"]
infer(tf.constant(4, dtype=float))

In [ ]:
infer.mutate()

In [ ]:
infer(tf.constant([4,5,6], dtype=float))

In [ ]:
module_multiple_signatures_path = os.path.join(model_dir, 'multiple_signatures')
call = module.__call__.get_concrete_function(tf.TensorSpec(None, tf.float32))
signatures = {"serving_default": call,
              "array_input": module.__call__.get_concrete_function(tf.TensorSpec([None], tf.float32))}

tf.saved_model.save(module, module_multiple_signatures_path, signatures=signatures)


In [ ]:
imported = tf.saved_model.load(module_multiple_signatures_path)
print(list(imported.signatures.keys()))
infer = imported.signatures["serving_default"]
print(infer(tf.constant([[3,3,4],[2,1,2]], dtype=float)))
infer = imported.signatures["array_input"]
print(infer(tf.constant([[3,3,4],[2,1,2]], dtype=float)))

In [ ]:
imported.mutate(tf.constant(4, dtype=float),tf.constant(4, dtype=float))